In [49]:
%matplotlib qt
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # sometimes needed to register 3D
import numpy as np
from matplotlib import colors
from matplotlib.widgets import Slider, Button




from medmnist import OrganMNIST3D

# Or NoduleMNIST3D, AdrenalMNIST3D, etc.
# Change these to match which dataset you've been assigned


train_dataset = OrganMNIST3D(split='train', size=28, download=True)
trainx = []
trainy = []

test_dataset = OrganMNIST3D(split='test', size=28, download=True)
testx = []
testy = []

val_dataset = OrganMNIST3D(split='train', size=28, download=True)
valx = []
valy = []

for i in range(len(train_dataset)):
    trainx.append(train_dataset[i][0])
    trainy.append(train_dataset[i][1])

for i in range(len(test_dataset)):
    testx.append(test_dataset[i][0])
    testy.append(test_dataset[i][1])

for i in range(len(val_dataset)):
    valx.append(val_dataset[i][0])
    valy.append(val_dataset[i][1])

#using float 32 for the time being because its quicker for cpus
trainx_tensor = tf.convert_to_tensor(trainx, dtype=tf.float16)
trainy_tensor = tf.convert_to_tensor(trainy, dtype=tf.float16)
#trainy_tensor = tf.convert_to_tensor(trainy, dtype=tf.float32) #since ours has multi class it needs to be 32
testx_tensor = tf.convert_to_tensor(testx, dtype=tf.float16)
testy_tensor = tf.convert_to_tensor(testy, dtype=tf.float16)
valx_tensor = tf.convert_to_tensor(valx, dtype=tf.float16)
valy_tensor = tf.convert_to_tensor(valy, dtype=tf.float16)
# float16 doesn't run any faster on the 4090s, but it cuts memory usage in half!


In [59]:

def myNet():

    model = tf.keras.Sequential(layers = [
        # #images only have 1 color scale (greyscale)
        tf.keras.layers.InputLayer(shape=(1, 28, 28, 28)),

        #(3, 3, 3) slides a 3x3x3 cube over the height, depth, width
        # can change bias to be 0.01 if the learning is very slow, helps kickstart it if needed later
        # Using data_format channels first because the input has the channels(axis here) first instead of last
        tf.keras.layers.Conv3D(32, (3, 3, 3), activation= 'relu', bias_initializer='zeros', data_format='channels_first'),
        tf.keras.layers.Conv3D(32, (3, 3, 3), activation= 'relu', bias_initializer='zeros', data_format='channels_first'),
        #pool size of (2, 2, 2) changes the input shape of (28, 28, 28, 1) to (14, 14, 14, 1)
        tf.keras.layers.MaxPooling3D(pool_size=(2,2,2)),
        #tf.keras.layers.AveragePooling3D(pool_size=(2,2,2)),


        #closing layers, copied from previous nets and can be updated
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(11, activation='softmax', name= 'output') #needs to be 11 because there are 10 classifications
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [62]:
model = myNet()
training_history = model.fit(
    trainx_tensor, trainy_tensor,
    batch_size=32, 
    epochs= 3)
    

Epoch 1/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.2698 - loss: 2.9292
Epoch 2/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.6148 - loss: 1.2952
Epoch 3/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.7322 - loss: 0.8525


In [63]:
test_loss, test_acc = model.evaluate(testx_tensor, testy_tensor)
print(f"Test Accuracy: {round(test_acc * 100, 2)}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 338ms/step - accuracy: 0.7148 - loss: 0.8744
Test Accuracy: 71.48%


max pooling:
Epoch 1/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2667 - loss: 2.3477
Epoch 2/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.5654 - loss: 1.3226
Epoch 3/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.7683 - loss: 0.7938

20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 361ms/step - accuracy: 0.7016 - loss: 0.9283
Test Accuracy: 70.16%


average pooling:
Epoch 1/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.2266 - loss: 2.3036
Epoch 2/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.5283 - loss: 1.4193
Epoch 3/3
31/31 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.6941 - loss: 0.9188

20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 354ms/step - accuracy: 0.7180 - loss: 1.0014
Test Accuracy: 71.8%